In [211]:
# Dependencies
import requests
import json
import pandas as pd

# Google developer API key
from config import gkey

In [213]:
addresses = pd.read_csv('inmuebles24-edo_Venta_test.csv')
display(addresses.head())
display(addresses.count())

,Unnamed: 0,name,location,description,link,price,operation,rooms,bathrooms,construction (m2),terrain (m2)
0,0,"Sierra Gorda, Lomas de Chapultepec","Calle Sierra gorda, Lomas de Chapultepec, Migu...",La exclusiva zona residencial de Lomas de Chap...,/propiedades/sierra-gorda-lomas-de-chapultepec...,"MN 25,000,000",venta,2.5,NaN,282.0,416.5
1,1,Enquentro Residencial Insurgentes Mixcoac,"Patriotismo 883, Insurgentes Mixcoac, Benito J...",Enquentro un proyecto moderno y singular. Ven ...,/propiedades/enquentro-residencial-insurgentes...,"MN 5,523,587",venta,2.5,NaN,140.5,140.5
2,2,Sky Tower Lindavista,"Av. Insurgentes Norte. 1346, Lindavista, Ciud...",Sky tower lindavista es un desarrollo de 184 d...,/propiedades/sky-tower-lindavista-54456938.html,"MN 3,210,000",venta,2.0,NaN,79.5,122.0
3,3,Musset 326,"Alfredo Musset 326, Polanco, Miguel Hidalgo",Atractivo e innovador proyecto residencial en ...,/propiedades/musset-326-56045986.html,"MN 20,098,744",venta,2.5,NaN,294.0,328.0
4,4,Progreso 40,"PROGRESO 40, Escandón, Miguel Hidalgo","Super oportunidad, aprovecha un 8% de descuent...",/propiedades/progreso-40-57552849.html,"MN 4,488,000",venta,2.0,NaN,114.0,161.0


Unnamed: 0           1000
name                 1000
location             1000
description          1000
link                 1000
price                1000
operation            1000
rooms                 981
bathrooms             819
construction (m2)     958
terrain (m2)          957
dtype: int64

In [214]:
#input the address
addressinput = addresses["location"].tolist()
print(addressinput)

['Calle Sierra gorda, Lomas de Chapultepec, Miguel Hidalgo', 'Patriotismo 883, Insurgentes Mixcoac, Benito Juárez', 'Av. Insurgentes Norte. 1346, Lindavista,  Ciudad de México (A lado del Costco Lindavista), Lindavista, Gustavo A. Madero', 'Alfredo Musset 326, Polanco, Miguel Hidalgo', 'PROGRESO 40, Escandón, Miguel Hidalgo', 'Jardines EN LA Montaña, Residencia Frente AL Bosque, Jardines en la Montaña, Tlalpan', 'Calle de la Constitución 18, Escandón, Miguel Hidalgo', 'Ures 87, Roma, Cuauhtémoc', 'Saratoga 605, Portales, Benito Juárez', 'Casas Grandes 149, Narvarte, Benito Juárez', 'Cóndor 260, Las Aguilas, Alvaro Obregón', 'Watteau 17, Santa Maria Nonoalco, Benito Juárez', 'El Parque de Coyoacán, Coyoacán', 'Casas Grandes 149, Narvarte, Benito Juárez', 'Paz Montes de Oca 96, General Pedro Maria Anaya, Benito Juárez', 'Saratoga 605 , Portales, Benito Juárez', 'Calle Musset, Garden House en Venta o Renta (js), Polanco I Sección, Miguel Hidalgo', 'Jaime Balmes, Departamento en Venta, Pol

In [215]:
# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

params = {"address": addressinput, "key": gkey}

#loop to get all locations from locaitions

lat      =[]
lng      =[]
calle    =[]
numero   =[]
colonia  =[]
ciudad   =[]
estado   =[]
CP       =[]
formatad =[]
locfound =[] #locations found
notfound =[] #locations not found

for index in params["address"]:
    #make request and print url
    try:
        response = requests.get(base_url, params={"address":index,"key": gkey})
        geo      = response.json()
        
        lat.append(geo["results"][0]["geometry"]["location"]["lat"])
        lng.append(geo["results"][0]["geometry"]["location"]["lng"])
        
        #sacar el número
        j=0
        bandera=0
        for results in geo["results"][0]["address_components"]:
            if geo["results"][0]["address_components"][j]["types"][0]=="street_number":
                numero.append(geo["results"][0]["address_components"][j]["long_name"])
                #print(f"{j} opción")
                #print(numero)
                bandera=1
            if j==len(geo["results"][0]["address_components"])-1 and bandera==0:
                numero.append(None)
                #print("noencontrado")
            j=j+1
        
        #sacar la calle
        j=0
        bandera=0
        for results in geo["results"][0]["address_components"]:
            if geo["results"][0]["address_components"][j]["types"][0]=="route":
                calle.append(geo["results"][0]["address_components"][j]["long_name"])
                #print(f"{j} opción")
                #print(calle)
                bandera=1
            if j==len(geo["results"][0]["address_components"])-1 and bandera==0:
                calle.append(None)
                #print("noencontrado")
            j=j+1
              
        #sacar la colonia
        j=0
        bandera=0
        for results in geo["results"][0]["address_components"]:
            if geo["results"][0]["address_components"][j]["types"][0]=="political" and geo["results"][0]["address_components"][j]["types"][1]=="sublocality":
                colonia.append(geo["results"][0]["address_components"][j]["long_name"])
                #print(f"{j} opción")
                #print(colonia)
                bandera=1
            if j==len(geo["results"][0]["address_components"])-1 and bandera==0:
                colonia.append(None)
                #print("noencontrado")
            j=j+1
                
        #sacar ciudad
        j=0
        bandera=0
        for results in geo["results"][0]["address_components"]:
            if geo["results"][0]["address_components"][j]["types"][0]=="locality" and geo["results"][0]["address_components"][j]["types"][1]=="political":
                ciudad.append(geo["results"][0]["address_components"][j]["long_name"])
                #print(f"{j} opción")
                #print(ciudad)
                bandera=1
            if j==len(geo["results"][0]["address_components"])-1 and bandera==0:
                ciudad.append(None)
                #print("noencontrado")
            j=j+1
                
        #Sacar Código Postal
        j=0
        bandera=0
        for results in geo["results"][0]["address_components"]:
            if geo["results"][0]["address_components"][j]["types"][0]=="postal_code" :
                CP.append(geo["results"][0]["address_components"][j]["long_name"])
                #print(f"{j} opción")
                #print(CP)
                bandera=1
            if j==len(geo["results"][0]["address_components"])-1 and bandera==0:
                CP.append(None)
                #print("noencontrado")
            j=j+1
              
        formatad.append(geo["results"][0]["formatted_address"])
        locfound.append(index)        
           
    
    except (KeyError, IndexError):
        notfound.append(index)

In [227]:
googleapivtacdmx = pd.DataFrame({
    'lat'      : lat,
    'lng'      : lng,
    'calle'    : calle,
    'numero'   : numero,
    'colonia'  : colonia,
    'ciudad'   : ciudad,
    #'estado'   : estado,
    'CP'       : CP,
    'formatad' : formatad,
    'location' : locfound
})
googleapivtacdmx

,lat,lng,calle,numero,colonia,ciudad,CP,formatad,location
0,19.421469,-99.225289,Sierra Gorda,None,Lomas de Chapultepec,Ciudad de México,None,"Sierra Gorda, Lomas de Chapultepec, Ciudad de ...","Calle Sierra gorda, Lomas de Chapultepec, Migu..."
1,19.373451,-99.187033,Avenida Patriotismo,883,Insurgentes Mixcoac,Ciudad de México,03920,"Avenida Patriotismo 883, Insurgentes Mixcoac, ...","Patriotismo 883, Insurgentes Mixcoac, Benito J..."
2,19.481763,-99.129618,Avenida Insurgentes Norte,1320,Magdalena de las Salinas,Ciudad de México,07760,"Av. Insurgentes Nte. 1320, Magdalena de las Sa...","Av. Insurgentes Norte. 1346, Lindavista, Ciud..."
3,19.436881,-99.196634,Alfredo de Musset,326,Polanco III Sección,Ciudad de México,11550,"A. Musset 326, Polanco, Polanco III Secc, Migu...","Alfredo Musset 326, Polanco, Miguel Hidalgo"
4,19.405612,-99.182466,Avenida Progreso,40,Escandón I Sección,Ciudad de México,11800,"Av Progreso 40, Escandón I Secc, Miguel Hidalg...","PROGRESO 40, Escandón, Miguel Hidalgo"
...,...,...,...,...,...,...,...,...,...
990,19.371435,-99.269843,Carretera México-Toluca,None,None,Ciudad de México,None,"Carr. México-Toluca, Ciudad de México, CDMX, M...","CARRETERA MÉXICO-TOLUCA, Contadero, Cuajimalpa..."
991,19.367424,-99.185752,None,None,San José Insurgentes,Mexico City,None,"San José Insurgentes, Mexico City, CDMX, Mexico","SAN Jose Insurgentes, San José Insurgentes, Be..."
992,19.386393,-99.225562,None,None,Santa Fe,Mexico City,None,"Santa Fe, Mexico City, CDMX, Mexico","Colonia Santa Fe Cuajimalpa, Santa Fe Cuajimal..."
993,19.424554,-99.222652,Monte Chimborazo,None,None,Ciudad de México,11000,"Monte Chimborazo, 11000 Ciudad de México, CDMX...","MONTE CHIMBORAZO, Lomas de Chapultepec, Miguel..."


In [228]:


display(len(lat))
display(len(lat))   
display(len(lng))   
display(len(calle))  
display(len(numero)) 
display(len(colonia))
display(len(ciudad))
#display(len(estado))
display(len(CP))
display(len(formatad))
display(len(locfound))
display(len(notfound))

995

995

995

995

995

...

995

995

995

995

5

In [229]:
print(notfound)

['Bosques de Reforma, Casa Sola a la Venta en Bosques de las Lomas (vw), Bosque de las Lomas, Miguel Hidalgo', 'Exclusiva!! Casa en Conjunto de Lujo a la Venta  Residencial Domus , Bosques de  las Lomas  (vw), Bosques de las Lomas, Cuajimalpa de Morelos', 'Increible Residencia en san Lorenzo Acopilco a la Venta (mc), San Lorenzo Acopilco, Cuajimalpa de Morelos', 'Calz. al Desierto de los Leones 5886, Olivar de los Padres., Olivar de los Padres, Alvaro Obregón', 'Gutierrez Zamora al 200, Las Aguilas, Alvaro Obregón']


In [236]:
 # Print the json (pretty printed)
#print(json.dumps(geo, indent=4, sort_keys=True))

,Unnamed: 0,name,location,description,link,price,operation,rooms,bathrooms,construction (m2),terrain (m2)
0,0,"Sierra Gorda, Lomas de Chapultepec","Calle Sierra gorda, Lomas de Chapultepec, Migu...",La exclusiva zona residencial de Lomas de Chap...,/propiedades/sierra-gorda-lomas-de-chapultepec...,"MN 25,000,000",venta,2.5,NaN,282.0,416.5
1,1,Enquentro Residencial Insurgentes Mixcoac,"Patriotismo 883, Insurgentes Mixcoac, Benito J...",Enquentro un proyecto moderno y singular. Ven ...,/propiedades/enquentro-residencial-insurgentes...,"MN 5,523,587",venta,2.5,NaN,140.5,140.5
2,2,Sky Tower Lindavista,"Av. Insurgentes Norte. 1346, Lindavista, Ciud...",Sky tower lindavista es un desarrollo de 184 d...,/propiedades/sky-tower-lindavista-54456938.html,"MN 3,210,000",venta,2.0,NaN,79.5,122.0
3,3,Musset 326,"Alfredo Musset 326, Polanco, Miguel Hidalgo",Atractivo e innovador proyecto residencial en ...,/propiedades/musset-326-56045986.html,"MN 20,098,744",venta,2.5,NaN,294.0,328.0
4,4,Progreso 40,"PROGRESO 40, Escandón, Miguel Hidalgo","Super oportunidad, aprovecha un 8% de descuent...",/propiedades/progreso-40-57552849.html,"MN 4,488,000",venta,2.0,NaN,114.0,161.0
...,...,...,...,...,...,...,...,...,...,...,...
995,995,Departamento en Venta o Renta en Be Grand Cont...,"CARRETERA MÉXICO-TOLUCA, Contadero, Cuajimalpa...",Excelente Oportunidad. Precioso departamento t...,/propiedades/departamento-en-venta-o-renta-en-...,"MN 5,700,000",venta,2.0,2.0,97.0,97.0
996,996,Departamento - San José Insurgentes,"SAN Jose Insurgentes, San José Insurgentes, Be...","PentHouse en San Jose Insurgentes, sobre Barra...",/propiedades/departamento-san-jose-insurgentes...,"MN 26,999,000",venta,3.0,3.0,363.0,363.0
997,997,Cad Punta del Parque Santa Fe Tb Ph1 2 Niveles...,"Colonia Santa Fe Cuajimalpa, Santa Fe Cuajimal...",Cad Punta del Parque Santa Fe. Torre B Pent Ho...,/propiedades/cad-punta-del-parque-santa-fe-tb-...,"USD 1,620,000",venta,3.0,3.0,560.0,560.0
998,998,Casa en Venta Lomas de Chapultepec,"MONTE CHIMBORAZO, Lomas de Chapultepec, Miguel...",Terreno regular. Con grande y Precioso jardín....,/propiedades/casa-en-venta-lomas-de-chapultepe...,"USD 2,150,000",venta,3.0,2.0,350.0,875.0


In [250]:
jointinfo_df = pd.merge(addresses, googleapivtacdmx, on="location" , how="left")
#jointinfo_df.set_index("location")
jointinfo_df.drop_duplicates(subset="location", keep="first", inplace=True)
jointinfo_df

,Unnamed: 0,name,location,description,link,price,operation,rooms,bathrooms,construction (m2),terrain (m2),lat,lng,calle,numero,colonia,ciudad,CP,formatad
0,0,"Sierra Gorda, Lomas de Chapultepec","Calle Sierra gorda, Lomas de Chapultepec, Migu...",La exclusiva zona residencial de Lomas de Chap...,/propiedades/sierra-gorda-lomas-de-chapultepec...,"MN 25,000,000",venta,2.5,NaN,282.0,416.5,19.421469,-99.225289,Sierra Gorda,None,Lomas de Chapultepec,Ciudad de México,None,"Sierra Gorda, Lomas de Chapultepec, Ciudad de ..."
1,1,Enquentro Residencial Insurgentes Mixcoac,"Patriotismo 883, Insurgentes Mixcoac, Benito J...",Enquentro un proyecto moderno y singular. Ven ...,/propiedades/enquentro-residencial-insurgentes...,"MN 5,523,587",venta,2.5,NaN,140.5,140.5,19.373451,-99.187033,Avenida Patriotismo,883,Insurgentes Mixcoac,Ciudad de México,03920,"Avenida Patriotismo 883, Insurgentes Mixcoac, ..."
2,2,Sky Tower Lindavista,"Av. Insurgentes Norte. 1346, Lindavista, Ciud...",Sky tower lindavista es un desarrollo de 184 d...,/propiedades/sky-tower-lindavista-54456938.html,"MN 3,210,000",venta,2.0,NaN,79.5,122.0,19.481763,-99.129618,Avenida Insurgentes Norte,1320,Magdalena de las Salinas,Ciudad de México,07760,"Av. Insurgentes Nte. 1320, Magdalena de las Sa..."
3,3,Musset 326,"Alfredo Musset 326, Polanco, Miguel Hidalgo",Atractivo e innovador proyecto residencial en ...,/propiedades/musset-326-56045986.html,"MN 20,098,744",venta,2.5,NaN,294.0,328.0,19.436881,-99.196634,Alfredo de Musset,326,Polanco III Sección,Ciudad de México,11550,"A. Musset 326, Polanco, Polanco III Secc, Migu..."
4,4,Progreso 40,"PROGRESO 40, Escandón, Miguel Hidalgo","Super oportunidad, aprovecha un 8% de descuent...",/propiedades/progreso-40-57552849.html,"MN 4,488,000",venta,2.0,NaN,114.0,161.0,19.405612,-99.182466,Avenida Progreso,40,Escandón I Sección,Ciudad de México,11800,"Av Progreso 40, Escandón I Secc, Miguel Hidalg..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2691,995,Departamento en Venta o Renta en Be Grand Cont...,"CARRETERA MÉXICO-TOLUCA, Contadero, Cuajimalpa...",Excelente Oportunidad. Precioso departamento t...,/propiedades/departamento-en-venta-o-renta-en-...,"MN 5,700,000",venta,2.0,2.0,97.0,97.0,19.371435,-99.269843,Carretera México-Toluca,None,None,Ciudad de México,None,"Carr. México-Toluca, Ciudad de México, CDMX, M..."
2692,996,Departamento - San José Insurgentes,"SAN Jose Insurgentes, San José Insurgentes, Be...","PentHouse en San Jose Insurgentes, sobre Barra...",/propiedades/departamento-san-jose-insurgentes...,"MN 26,999,000",venta,3.0,3.0,363.0,363.0,19.367424,-99.185752,None,None,San José Insurgentes,Mexico City,None,"San José Insurgentes, Mexico City, CDMX, Mexico"
2693,997,Cad Punta del Parque Santa Fe Tb Ph1 2 Niveles...,"Colonia Santa Fe Cuajimalpa, Santa Fe Cuajimal...",Cad Punta del Parque Santa Fe. Torre B Pent Ho...,/propiedades/cad-punta-del-parque-santa-fe-tb-...,"USD 1,620,000",venta,3.0,3.0,560.0,560.0,19.386393,-99.225562,None,None,Santa Fe,Mexico City,None,"Santa Fe, Mexico City, CDMX, Mexico"
2694,998,Casa en Venta Lomas de Chapultepec,"MONTE CHIMBORAZO, Lomas de Chapultepec, Miguel...",Terreno regular. Con grande y Precioso jardín....,/propiedades/casa-en-venta-lomas-de-chapultepe...,"USD 2,150,000",venta,3.0,2.0,350.0,875.0,19.424554,-99.222652,Monte Chimborazo,None,None,Ciudad de México,11000,"Monte Chimborazo, 11000 Ciudad de México, CDMX..."


In [252]:
jointinfo_df.to_csv("respuesta1000google.csv")